<a href="https://colab.research.google.com/github/CataSiches/Coder-Siches/blob/main/DataScienceII_Proyecto_Siches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Siches Catalina: Factores de impacto en el ingreso a la educación superior**

# Metadata

***Fuentes:***

Paises por Continente:

https://www.kaggle.com/datasets/hserdaraltan/countries-by-continent?resource=download

World Bank API

https://datahelpdesk.worldbank.org/knowledgebase/topics/125589-developer-information

Unesco

https://pypi.org/project/unesco-reader/

***Descripción:***

El dataset construido presenta datos sobre la matriculación a la educación superior y el PBI de determinadios paises en un rango de años en particular (1990 - 2021), así como el continente al que pertenecen, y la tasa de natalidad 19 años previos.

***Diccionario de Datos:***

Continente: Nombre del continente en el cual se encuentra el país.

Pais: Nombre del país.

Año: Año de los datos del PBI y Matriculas

PBI_PC_USD: PBI per capita en dolares del país en cuestión.

Porcentaje_Matriculas: tasa bruta de matriculación en educación terciaria o universitaria, expresada como el porcentaje del total de personas inscriptas (sin importar su edad) respecto a la población del grupo etario que oficialmente corresponde a la educación terciaria (aproximadamente 18-22 años).

Año_Natalidad: 19 años antes que los datos de matriculación

Tasa_Natalidad: Indicador demográfico que mide la cantidad de nacimientos vivos ocurridos en un año por cada 1.000 personas de la población total del país.

# Objetivo Analítico

***Objetivo:***

Entrenar un algoritmo para predecir matriculaciones futuras a la educación superior a partir de las tasas de natalidad y las projecciones de PBI per capita. Pudiendo de esta manera contar con una preparación adecuada para recibir al alumnado. Y a su vez, tomar las medidas que se consideren necesarias para aumentar la cantidad de matriculas pedidas.

***Contexto Comercial:***

El acceso a la educación superior favorece el desarrollo de los paises y genera avances en los mismos. Es por esto que es necesario analizar que lo impacta y como fortalecer la matriculación.

***Problema Comercial Contexto:***

Se observa que la natalidad ha disminuido a nivel global en los ultimos años, y a su vez las economias mundiales muestran inestabilidad y fluctuaciones. Ambos factores podrían ser influyentes en el porcentaje anual de matriculacion a la educación superior.

***Hipotesis:***

La disminución en la tasa de natalidad tiene un impacto en la cantidad de alumnos y alumnas que se matriculen, pero no en el porcentaje de matriculación, ya que el mismo depende de la cantidad de personas en edad para el ingreso a la educación superior.

El PBI per capita tiene un impacto directo en el porcentaje de matriculación, ya que cuando hay una estabilidad economica en la población no hay una necesidad general de que quienes finalizan el colegio tengan que pasar directo al trabajo, sino que pueden continuar su educación.

# Importación Librerias Generales

In [1]:
# Importar librerias
# Manipulación numerica
import pandas as pd
import numpy as np
# Visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# API acceso
import requests
# Time para trabajo con años
import time

# DataSet Paises por Continente

## Descarga Dataset

In [2]:
# Descargar DataBase Drive
# https://drive.google.com/file/d/1nQxYnT6Dz2Ay4qJV_HsHaNB1s0RF0xS0/view?usp=drive_link
# Source url: https://www.kaggle.com/datasets/hserdaraltan/countries-by-continent?resource=download
!gdown 1nQxYnT6Dz2Ay4qJV_HsHaNB1s0RF0xS0

Downloading...
From: https://drive.google.com/uc?id=1nQxYnT6Dz2Ay4qJV_HsHaNB1s0RF0xS0
To: /content/CountriesbyContinents.csv
100% 3.61k/3.61k [00:00<00:00, 11.3MB/s]


In [3]:
# DataFrame con DataBase
df_paises = pd.read_csv('CountriesbyContinents.csv')

# Chequeo descarga correcta
df_paises.head()


,Continent,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina


## Exploración Dataset Paises

In [4]:
# Información df_paises
df_paises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Continent  196 non-null    object
 1   Country    196 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [5]:
# Descripción
df_paises.describe()

,Continent,Country
count,196,196
unique,6,196
top,Africa,Algeria
freq,54,1


In [6]:
# Chequeo nulos
df_paises.isnull().sum()

,0
Continent,0
Country,0


In [7]:
# Continentes
df_paises['Continent'].unique()

array(['Africa', 'Asia', 'Europe', 'North America', 'Oceania',
       'South America'], dtype=object)

# DataFrame World Bank información paises

## Conección con API

In [8]:
# Conectar con Word Bank y descargar codigos de países
# A partir de esos codigos luego se podrá acceder a la información requerida
# Llamada a la API
url = "https://api.worldbank.org/v2/country?format=json&per_page=500"
response = requests.get(url)
data = response.json()[1]  # La lista de países está en la segunda posición

# Extraer solo las columnas deseadas
df_codes = pd.DataFrame([{
    "3_letter_code": c["id"],       # Código de 3 letras
    "2_letter_code": c["iso2Code"], # Código de 2 letras
    "name": c["name"]               # Nombre del país
} for c in data])

print(df_codes.head())

  3_letter_code 2_letter_code                         name
0           ABW            AW                        Aruba
1           AFE            ZH  Africa Eastern and Southern
2           AFG            AF                  Afghanistan
3           AFR            A9                       Africa
4           AFW            ZI   Africa Western and Central


## Exploración DataSet Codes

In [9]:
# Información df_codes
df_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   3_letter_code  296 non-null    object
 1   2_letter_code  296 non-null    object
 2   name           296 non-null    object
dtypes: object(3)
memory usage: 7.1+ KB


In [10]:
# Descripción
df_codes.describe()

,3_letter_code,2_letter_code,name
count,296,296,296
unique,296,296,296
top,ZWE,ZW,Zimbabwe
freq,1,1,1


In [11]:
# Chequeo Nulos
df_codes.isnull().sum()

,0
3_letter_code,0
2_letter_code,0
name,0


In [12]:
# Shape
df_codes.shape

(296, 3)

# Merge DataFrames

## Normalización

In [13]:
# Preparar dataframes para merge
df_paises["Country"] = df_paises["Country"].str.strip().str.title()
df_codes["name"] = df_codes["name"].str.strip().str.title()

In [14]:
# Verificar si los nombres de paises coinciden
# Conjuntos de países en cada df
paises_df1 = set(df_paises["Country"])
paises_df2 = set(df_codes["name"])

# Países que están en df1 pero no en df2
no_en_df2 = paises_df1 - paises_df2
print("En df1 pero no en df2:", no_en_df2)

# Países que están en df2 pero no en df1
no_en_df1 = paises_df2 - paises_df1
print("En df2 pero no en df1:", no_en_df1)

En df1 pero no en df2: {'Saint Vincent And The Grenadines', 'Laos', 'Vatican City', 'Burkina', 'Venezuela', 'Congo', 'Iran', 'Ivory Coast', 'Saint Kitts And Nevis', 'East Timor', 'Democratic Republic Of Congo', 'Russia', 'Swaziland', 'Brunei', 'Syria', 'Kyrgyzstan', 'Yemen', 'Gambia', 'Burma (Myanmar)', 'Bahamas', 'Vietnam', 'North Korea', 'Cape Verde', 'Egypt', 'Micronesia', 'South Korea', 'Turkey', 'Saint Lucia', 'Somalia', 'Taiwan', 'Hong Kong', 'Slovakia', 'Macedonia'}
En df2 pero no en df1: {'Sub-Saharan Africa (Ifc Classification)', 'Middle East, North Africa, Afghanistan & Pakistan (Ibrd Only)', 'Africa', 'Yemen, Rep.', 'Hong Kong Sar, China', 'Post-Demographic Dividend', 'Puerto Rico (Us)', 'Early-Demographic Dividend', 'Congo, Rep.', 'Bahamas, The', 'Brunei Darussalam', 'Middle East (Developing Only)', 'Least Developed Countries: Un Classification', 'Curacao', 'Ibrd, Including Blend', 'Europe & Central Asia (Excluding High Income)', 'North America', 'Ida Blend', 'East Asia & P

In [15]:
# Descargar librería para chequear likeness entre paises que no coinciden
!pip install thefuzz[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.0 MB/s eta 0:00:00


In [16]:
# Importar librería
from thefuzz import process

# Supongamos que estos son los países que no matchearon exacto
no_en_df2 = list(no_en_df2)

# Para cada país en df1 que no coincide, buscamos el más parecido en df2
for pais in no_en_df2[:20]:  # [:20] para no imprimir miles
    mejor_match = process.extractOne(pais, paises_df2)
    print(f"{pais:30}  -->  {mejor_match}")


Saint Vincent And The Grenadines  -->  ('St. Vincent And The Grenadines', 94)
Laos                            -->  ('Lao Pdr', 77)
Vatican City                    -->  ('Italy', 68)
Burkina                         -->  ('Burkina Faso', 90)
Venezuela                       -->  ('Venezuela, Rb', 95)
Congo                           -->  ('Congo, Rep.', 90)
Iran                            -->  ('Iran, Islamic Rep.', 90)
Ivory Coast                     -->  ("Cote D'Ivoire", 63)
Saint Kitts And Nevis           -->  ('St. Kitts And Nevis', 90)
East Timor                      -->  ('Middle East, North Africa, Afghanistan & Pakistan (Ibrd Only)', 86)
Democratic Republic Of Congo    -->  ('Congo, Rep.', 86)
Russia                          -->  ('Russian Federation', 90)
Swaziland                       -->  ('Poland', 72)
Brunei                          -->  ('Brunei Darussalam', 90)
Syria                           -->  ('Syrian Arab Republic', 90)
Kyrgyzstan                      -->  ('Kyrgyz R

In [17]:
# Armar diccionario para modificar nombres en dataframe

reemplazos = {
    # confiables (score >= 90)
    "Iran": "Iran, Islamic Rep.",
    "Saint Vincent And The Grenadines": "St. Vincent And The Grenadines",
    "Yemen": "Yemen, Rep.",
    "Venezuela": "Venezuela, Rb",
    "Bahamas": "Bahamas, The",
    "Gambia": "Gambia, The",
    "Burkina": "Burkina Faso",
    "Vietnam": "Viet Nam",
    "Saint Kitts And Nevis": "St. Kitts And Nevis",

    # casos que necesitan ajuste manual
    "Slovakia": "Slovak Republic",
    "North Korea": "Korea, Dem. People'S Rep.",
    "Turkey": "Turkiye",
    "Kyrgyzstan": "Kyrgyz Republic",
    "Ivory Coast": "Cote D'Ivoire",

    # casos incorrectos en fuzzy → corrección manual
    "Vatican City": "Holy See",       # así suele aparecer en World Bank
    "East Timor": "Timor-Leste",
    "Saint Lucia": "St. Lucia",
    "Taiwan": "Taiwan, China",        # en World Bank lo listan así, si aparece
    "Democratic Republic Of Congo": "Congo, Dem. Rep."
}

In [18]:
# Reemplazar
df_paises["Country_normalizado"] = df_paises["Country"].replace(reemplazos)

In [19]:
df_paises.head()

,Continent,Country,Country_normalizado
0,Africa,Algeria,Algeria
1,Africa,Angola,Angola
2,Africa,Benin,Benin
3,Africa,Botswana,Botswana
4,Africa,Burkina,Burkina Faso


In [20]:
# Nulos
df_paises.isnull().sum()

,0
Continent,0
Country,0
Country_normalizado,0


## Merge

In [21]:
# Merge df_paises y df_codes
# Columnas: Continent, Country_normalizado, 3_letter_code, 2_letter_code
df_merge = pd.merge(df_paises, df_codes, left_on="Country_normalizado", right_on="name", how="left")
df_merge = df_merge[["Continent", "Country_normalizado", "3_letter_code", "2_letter_code"]]

## Chequeo de nuevo dataframe

In [22]:
df_merge.head()

,Continent,Country_normalizado,3_letter_code,2_letter_code
0,Africa,Algeria,DZA,DZ
1,Africa,Angola,AGO,AO
2,Africa,Benin,BEN,BJ
3,Africa,Botswana,BWA,BW
4,Africa,Burkina Faso,BFA,BF


In [23]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Continent            196 non-null    object
 1   Country_normalizado  196 non-null    object
 2   3_letter_code        180 non-null    object
 3   2_letter_code        180 non-null    object
dtypes: object(4)
memory usage: 6.3+ KB


In [24]:
# Check null
df_merge.isnull().sum()

,0
Continent,0
Country_normalizado,0
3_letter_code,16
2_letter_code,16


In [25]:
# Observar NULL datos
df_merge[df_merge.isnull().any(axis=1)]

,Continent,Country_normalizado,3_letter_code,2_letter_code
7,Africa,Cape Verde,NaN,NaN
11,Africa,Congo,NaN,NaN
14,Africa,Egypt,NaN,NaN
43,Africa,Somalia,NaN,NaN
47,Africa,Swaziland,NaN,NaN
58,Asia,Brunei,NaN,NaN
59,Asia,Burma (Myanmar),NaN,NaN
63,Asia,Hong Kong,NaN,NaN
73,Asia,South Korea,NaN,NaN
76,Asia,Laos,NaN,NaN


## Trabajo con Nulos

In [26]:
# Busqueda y reemplazo manual de nombre de paises con valores NaN

reemplazos.update({
    "Cape Verde": "Cabo Verde",
    "Congo": "Congo, Rep.",
    "Egypt": "Egypt, Arab Rep.",
    "Swaziland": "Eswatini",
    "Brunei": "Brunei Darussalam",
    "Burma (Myanmar)": "Myanmar",
    "Hong Kong": "Hong Kong SAR, China",
    "South Korea": "Korea, Rep.",
    "Laos": "Lao PDR",
    "Russia": "Russian Federation",
    "Syria": "Syrian Arab Republic",
    "Macedonia": "North Macedonia",
    "Micronesia": "Micronesia, Fed. Sts."
})

In [27]:
df_paises["Country_normalizado"] = df_paises["Country"].replace(reemplazos)

In [28]:
df_merge2 = pd.merge(
    df_paises, df_codes,
    left_on="Country_normalizado", right_on="name",
    how="left"
)[["Continent", "Country_normalizado", "3_letter_code", "2_letter_code"]]

In [29]:
df_merge2.head()

,Continent,Country_normalizado,3_letter_code,2_letter_code
0,Africa,Algeria,DZA,DZ
1,Africa,Angola,AGO,AO
2,Africa,Benin,BEN,BJ
3,Africa,Botswana,BWA,BW
4,Africa,Burkina Faso,BFA,BF


In [30]:
# Observar Cantidad valores nulos
df_merge2.isnull().sum()

,0
Continent,0
Country_normalizado,0
3_letter_code,5
2_letter_code,5


In [31]:
# Paises con valores nulos
df_merge2[df_merge2.isnull().any(axis=1)]

,Continent,Country_normalizado,3_letter_code,2_letter_code
43,Africa,Somalia,NaN,NaN
63,Asia,"Hong Kong SAR, China",NaN,NaN
76,Asia,Lao PDR,NaN,NaN
91,Asia,"Taiwan, China",NaN,NaN
146,Europe,Holy See,NaN,NaN


In [32]:
# Find Somalia in df_codes, 3_letter_code is SOM
df_codes[df_codes["name"].str.contains("Somalia", case=False)]

,3_letter_code,2_letter_code,name
241,SOM,SO,"Somalia, Fed. Rep."


In [33]:
# Reemplazar 3_letter_code y 2_letter_code de Somalia con los datos encontrados
df_merge2.loc[df_merge2["Country_normalizado"] == "Somalia",
              ["3_letter_code", "2_letter_code"]] = ["SOM", "SO"]

In [34]:
# Find Hong Kong in df_codes
df_codes[df_codes["name"].str.contains("Hong Kong", case=False)]

,3_letter_code,2_letter_code,name
118,HKG,HK,"Hong Kong Sar, China"


In [35]:
# Reemplazar 3_letter_code y 2_letter_code de HongKong con los datos encontrados
df_merge2.loc[df_merge2["Country_normalizado"] == "Hong Kong SAR, China",
              ["3_letter_code", "2_letter_code"]] = ["HKG", "HK"]

In [36]:
# Find Lao in df_codes
df_codes[df_codes["name"].str.contains("Lao", case=False)]

,3_letter_code,2_letter_code,name
152,LAO,LA,Lao Pdr


In [37]:
# Reemplazar 3_letter_code y 2_letter_code de Lao con los datos encontrados
df_merge2.loc[df_merge2["Country_normalizado"] == "Lao PDR",
              ["3_letter_code", "2_letter_code"]] = ["LAO", "LA"]

In [38]:
# Find Taiwan in df_codes
df_codes[df_codes["name"].str.contains("Taiwan", case=False)]

,3_letter_code,2_letter_code,name


In [39]:
# Find Holy See in df_codes
df_codes[df_codes["name"].str.contains("Holy", case=False)]

,3_letter_code,2_letter_code,name


In [40]:
# Eliminar Taiwan y Holy See de df_merge2
df_merge2 = df_merge2[~df_merge2["Country_normalizado"].isin(["Taiwan, China", "Holy See"])]

In [41]:
# Rechequear nulos df_merge2
df_merge2.isnull().sum()

,0
Continent,0
Country_normalizado,0
3_letter_code,0
2_letter_code,0


In [42]:
# Ver Shape merge2
df_merge2.shape

(194, 4)

In [43]:
# Ver Namibia
df_merge2[df_merge2["Country_normalizado"] == "Namibia"]

,Continent,Country_normalizado,3_letter_code,2_letter_code
35,Africa,Namibia,NAM,NA


## Guardado de DataFrame en .csv

In [44]:
# Guardar dataframe en archivo csv
df_merge2.to_csv("codigo_paises.csv", index=False)

# Años a explorar

In [45]:
# Años de interes: 1990 al 2020
años = list(range(1990, 2021))
años

[1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]

# Merge Años en df

In [46]:
# Transformar lista en df
df_años = pd.DataFrame(años, columns=["Años"])
df_años.head()

,Años
0,1990
1,1991
2,1992
3,1993
4,1994


In [47]:
# DataFrame codigo_paises.csv
df_c_paises = pd.read_csv("codigo_paises.csv")
df_c_paises.head()

,Continent,Country_normalizado,3_letter_code,2_letter_code
0,Africa,Algeria,DZA,DZ
1,Africa,Angola,AGO,AO
2,Africa,Benin,BEN,BJ
3,Africa,Botswana,BWA,BW
4,Africa,Burkina Faso,BFA,BF


In [48]:
# Nulos
df_c_paises.isnull().sum()

,0
Continent,0
Country_normalizado,0
3_letter_code,0
2_letter_code,1


In [49]:
# Mostrar 2_letter_code nulo
df_c_paises[df_c_paises["2_letter_code"].isnull()]

,Continent,Country_normalizado,3_letter_code,2_letter_code
35,Africa,Namibia,NAM,NaN


In [50]:
# Creo que porque el codigo de Namibia es NA lo toma como nulo, cambiarlo
df_c_paises.loc[df_c_paises["Country_normalizado"] == "Namibia", "2_letter_code"] = "NA"

In [51]:
# Combinar con df_c_paises
df_full = (df_c_paises.assign(key=1)
                        .merge(df_años.assign(key=1), on="key")
                        .drop("key", axis=1))
df_full.head()

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años
0,Africa,Algeria,DZA,DZ,1990
1,Africa,Algeria,DZA,DZ,1991
2,Africa,Algeria,DZA,DZ,1992
3,Africa,Algeria,DZA,DZ,1993
4,Africa,Algeria,DZA,DZ,1994


In [52]:
df_full.shape

(6014, 5)

In [53]:
# Nulos
df_full.isnull().sum()

,0
Continent,0
Country_normalizado,0
3_letter_code,0
2_letter_code,0
Años,0


# Acceso API World Bank descarga historica PBI por país

In [54]:
# Transformar codigos de dos letras en lista para hacer un loop
paises = df_merge2["2_letter_code"].dropna().unique().tolist()

# Transformar lista de años en rango
años = list(range(1990, 2021))

# Indicador para acceder al PBI de cada país
indicador = "NY.GDP.PCAP.CD"

# Definir lista para guardar resultados
rows = []

# Calcular total de paises para establerer un porcentaje de progreso
total = len(paises)

# Loop
for i, codigo in enumerate(paises, 1):  # enumerate para saber en qué país vas
    # Url conección, necesita el codigo del páis, el indicador de la información que se busca y el rango de años
    url = f"https://api.worldbank.org/v2/country/{codigo}/indicator/{indicador}?date=1990:2020&format=json&per_page=1000"
    response = requests.get(url)

    # Asegurarse una conección estable
    if response.status_code == 200:
        # Almacenamiento de los datos en lenguaje json
        data = response.json()
        # Si hay datos
        if len(data) > 1:
            # Por cada entrada en los datos
            for entry in data[1]:
                # si el valor no es nulo
                if entry["value"] is not None:
                    # agregar datos a la lista row
                    rows.append({
                        "codigo_pais": codigo,
                        "año": int(entry["date"]),
                        "PBI_USD_PerCapita": entry["value"]
                    })
    else:
        print(f"Error for {codigo}: {response.status_code}")

     # Mostrar progreso (cada 10 países aprox.)
    if i % 10 == 0 or i == total:
        progreso = (i / total) * 100
        print(f"Progreso: {i}/{total} países ({progreso:.1f}%) completado")

    time.sleep(0.5)  # evitar limites

Progreso: 10/194 países (5.2%) completado
Progreso: 20/194 países (10.3%) completado
Progreso: 30/194 países (15.5%) completado
Progreso: 40/194 países (20.6%) completado
Progreso: 50/194 países (25.8%) completado
Progreso: 60/194 países (30.9%) completado
Progreso: 70/194 países (36.1%) completado
Progreso: 80/194 países (41.2%) completado
Progreso: 90/194 países (46.4%) completado
Progreso: 100/194 países (51.5%) completado
Progreso: 110/194 países (56.7%) completado
Progreso: 120/194 países (61.9%) completado
Progreso: 130/194 países (67.0%) completado
Progreso: 140/194 países (72.2%) completado
Progreso: 150/194 países (77.3%) completado
Progreso: 160/194 países (82.5%) completado
Progreso: 170/194 países (87.6%) completado
Progreso: 180/194 países (92.8%) completado
Progreso: 190/194 países (97.9%) completado
Progreso: 194/194 países (100.0%) completado


In [55]:
# Chequeo de información guardada
# Ver primeros 10 items en rows
rows[:10]

[{'codigo_pais': 'DZ', 'año': 2020, 'PBI_USD_PerCapita': 3743.5419522929},
 {'codigo_pais': 'DZ', 'año': 2019, 'PBI_USD_PerCapita': 4468.45341883656},
 {'codigo_pais': 'DZ', 'año': 2018, 'PBI_USD_PerCapita': 4577.21029180491},
 {'codigo_pais': 'DZ', 'año': 2017, 'PBI_USD_PerCapita': 4554.66753957828},
 {'codigo_pais': 'DZ', 'año': 2016, 'PBI_USD_PerCapita': 4424.98529027556},
 {'codigo_pais': 'DZ', 'año': 2015, 'PBI_USD_PerCapita': 4685.05902729002},
 {'codigo_pais': 'DZ', 'año': 2014, 'PBI_USD_PerCapita': 6094.69392314956},
 {'codigo_pais': 'DZ', 'año': 2013, 'PBI_USD_PerCapita': 5979.60138960585},
 {'codigo_pais': 'DZ', 'año': 2012, 'PBI_USD_PerCapita': 6033.64884689543},
 {'codigo_pais': 'DZ', 'año': 2011, 'PBI_USD_PerCapita': 5916.31364364343}]

In [56]:
# Transformar en dataframe
df_pbi = pd.DataFrame(rows)
df_pbi.head()

,codigo_pais,año,PBI_USD_PerCapita
0,DZ,2020,3743.541952
1,DZ,2019,4468.453419
2,DZ,2018,4577.210292
3,DZ,2017,4554.667540
4,DZ,2016,4424.985290


In [57]:
# Chequeo Nulos
df_pbi.isnull().sum()

,0
codigo_pais,0
año,0
PBI_USD_PerCapita,0


In [58]:
# Shape
df_pbi.shape

(5898, 3)

In [59]:
# Redondear PBI_USD_PerCapita por dos decimales
df_pbi["PBI_USD_PC_formatted"] = df_pbi["PBI_USD_PerCapita"].round(2)
df_pbi.head()

,codigo_pais,año,PBI_USD_PerCapita,PBI_USD_PC_formatted
0,DZ,2020,3743.541952,3743.54
1,DZ,2019,4468.453419,4468.45
2,DZ,2018,4577.210292,4577.21
3,DZ,2017,4554.667540,4554.67
4,DZ,2016,4424.985290,4424.99


In [60]:
# Transformar en .csv
df_pbi.to_csv("pbi_per_capita.csv", index=False)

# Merge DataFrames

## Merge y exploración rápida

In [61]:
# Bajar pbi_per_capita.csv a df
df_pbi2 = pd.read_csv("pbi_per_capita.csv")
df_pbi2.head()

,codigo_pais,año,PBI_USD_PerCapita,PBI_USD_PC_formatted
0,DZ,2020,3743.541952,3743.54
1,DZ,2019,4468.453419,4468.45
2,DZ,2018,4577.210292,4577.21
3,DZ,2017,4554.667540,4554.67
4,DZ,2016,4424.985290,4424.99


In [62]:
# Combinar df_full con df_pbi
temp_df_full = pd.merge(df_full, df_pbi2, left_on=["2_letter_code", "Años"], right_on=["codigo_pais", "año"], how="left")
# Dejar solo columnas de interes
df_full_pbi = temp_df_full[["Continent", "Country_normalizado", "3_letter_code", "2_letter_code", "Años", "PBI_USD_PC_formatted"]]
# Renombrar Columna PBI
df_full_pbi = df_full_pbi.rename(columns={"PBI_USD_PC_formatted": "PIB_per_capita_USD"})

df_full_pbi.head()

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
0,Africa,Algeria,DZA,DZ,1990,2445.18
1,Africa,Algeria,DZA,DZ,1991,1759.11
2,Africa,Algeria,DZA,DZ,1992,1802.69
3,Africa,Algeria,DZA,DZ,1993,1831.05
4,Africa,Algeria,DZA,DZ,1994,1525.54


In [63]:
# Shape
df_full_pbi.shape

(6014, 6)

In [64]:
# Chequeo Nulos
df_full_pbi.isnull().sum()

,0
Continent,0
Country_normalizado,0
3_letter_code,0
2_letter_code,0
Años,0
PIB_per_capita_USD,147


In [65]:
# Cantidad de registros por país
df_full_pbi["Country_normalizado"].value_counts()

,count
Country_normalizado,
Algeria,31
Angola,31
Benin,31
Botswana,31
Burkina Faso,31
...,...
Paraguay,31
Peru,31
Suriname,31


## Trabajo con nulos

### Observación

In [66]:
# Observar Nulos
df_full_pbi[df_full_pbi.isnull().any(axis=1)]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
496,Africa,Eritrea,ERI,ER,1990,NaN
497,Africa,Eritrea,ERI,ER,1991,NaN
518,Africa,Eritrea,ERI,ER,2012,NaN
519,Africa,Eritrea,ERI,ER,2013,NaN
520,Africa,Eritrea,ERI,ER,2014,NaN
...,...,...,...,...,...,...
6009,South America,"Venezuela, Rb",VEN,VE,2016,NaN
6010,South America,"Venezuela, Rb",VEN,VE,2017,NaN
6011,South America,"Venezuela, Rb",VEN,VE,2018,NaN
6012,South America,"Venezuela, Rb",VEN,VE,2019,NaN


In [67]:
# Pasar Nulos a df
df_nulos = df_full_pbi[df_full_pbi.isnull().any(axis=1)]
df_nulos.shape

(147, 6)

In [68]:
# Nombres de paises unicos
df_nulos["Country_normalizado"].unique()

array(['Eritrea', 'Mozambique', 'Namibia', 'South Sudan', 'Afghanistan',
       "Korea, Dem. People'S Rep.", 'Yemen, Rep.', 'Estonia', 'Latvia',
       'Lithuania', 'Montenegro', 'San Marino', 'Serbia', 'Venezuela, Rb'],
      dtype=object)

In [69]:
# Cantidad de rows por pais
df_nulos["Country_normalizado"].value_counts()

,count
Country_normalizado,
"Korea, Dem. People'S Rep.",31
Namibia,31
South Sudan,23
Eritrea,11
Afghanistan,10
San Marino,7
Montenegro,7
"Venezuela, Rb",6
Latvia,5


Observando la cantidad de nulos por país, creo que si el porcentaje de nulos supera 20 del total creo conveniente retirar ese país como posible sujeto de investigación.

### Eliminación Paises con 20% o más de nulos

In [70]:
# Cantidad de paises df_full_pbi
df_full_pbi["Country_normalizado"].nunique()

194

In [71]:
# Cantidad total de años por país
total_years = df_full_pbi["Años"].nunique()

# Calcular % de NaN por país
nan_ratio = (
    df_full_pbi.groupby("Country_normalizado")["PIB_per_capita_USD"]
    .apply(lambda x: x.isna().mean())
)

# Países con menos del 20% de NaN
valid_countries = nan_ratio[nan_ratio < 0.2].index

# Filtrar el DataFrame
df_full_pbi_clean = df_full_pbi[df_full_pbi["Country_normalizado"].isin(valid_countries)]

In [72]:
df_full_pbi_clean.head()

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
0,Africa,Algeria,DZA,DZ,1990,2445.18
1,Africa,Algeria,DZA,DZ,1991,1759.11
2,Africa,Algeria,DZA,DZ,1992,1802.69
3,Africa,Algeria,DZA,DZ,1993,1831.05
4,Africa,Algeria,DZA,DZ,1994,1525.54


In [73]:
df_full_pbi_clean.shape

(5797, 6)

In [74]:
# Observar cantidad de paises
df_full_pbi_clean["Country_normalizado"].nunique()

187

In [75]:
# Chequear si Venezuela sigue estando
df_full_pbi_clean[df_full_pbi_clean["Country_normalizado"] == "Venezuela, Rb"]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
5983,South America,"Venezuela, Rb",VEN,VE,1990,2451.55
5984,South America,"Venezuela, Rb",VEN,VE,1991,2632.54
5985,South America,"Venezuela, Rb",VEN,VE,1992,2907.30
5986,South America,"Venezuela, Rb",VEN,VE,1993,2824.69
5987,South America,"Venezuela, Rb",VEN,VE,1994,2688.98
5988,South America,"Venezuela, Rb",VEN,VE,1995,3486.98
5989,South America,"Venezuela, Rb",VEN,VE,1996,3112.83
5990,South America,"Venezuela, Rb",VEN,VE,1997,3710.83
5991,South America,"Venezuela, Rb",VEN,VE,1998,3869.98
5992,South America,"Venezuela, Rb",VEN,VE,1999,4070.91


In [76]:
# Chequear si Montenegro sigue en el df
df_full_pbi_clean[df_full_pbi_clean["Country_normalizado"] == "Montenegro"]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD


### Analizar caso a caso el resto de nulos

In [77]:
# Observar datos Venezuela
df_nulos[df_nulos["Country_normalizado"] == "Venezuela, Rb"]


,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
6008,South America,"Venezuela, Rb",VEN,VE,2015,NaN
6009,South America,"Venezuela, Rb",VEN,VE,2016,NaN
6010,South America,"Venezuela, Rb",VEN,VE,2017,NaN
6011,South America,"Venezuela, Rb",VEN,VE,2018,NaN
6012,South America,"Venezuela, Rb",VEN,VE,2019,NaN
6013,South America,"Venezuela, Rb",VEN,VE,2020,NaN


In [78]:
# Observar datos Latvia
df_nulos[df_nulos["Country_normalizado"] == "Latvia"]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
3782,Europe,Latvia,LVA,LV,1990,NaN
3783,Europe,Latvia,LVA,LV,1991,NaN
3784,Europe,Latvia,LVA,LV,1992,NaN
3785,Europe,Latvia,LVA,LV,1993,NaN
3786,Europe,Latvia,LVA,LV,1994,NaN


In [79]:
# Observar datos Lithuania
df_nulos[df_nulos["Country_normalizado"] == "Lithuania"]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
3844,Europe,Lithuania,LTU,LT,1990,NaN
3845,Europe,Lithuania,LTU,LT,1991,NaN
3846,Europe,Lithuania,LTU,LT,1992,NaN
3847,Europe,Lithuania,LTU,LT,1993,NaN
3848,Europe,Lithuania,LTU,LT,1994,NaN


In [80]:
# Observar datos Serbia
df_nulos[df_nulos["Country_normalizado"] == "Serbia"]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
4247,Europe,Serbia,SRB,RS,1990,NaN
4248,Europe,Serbia,SRB,RS,1991,NaN
4249,Europe,Serbia,SRB,RS,1992,NaN
4250,Europe,Serbia,SRB,RS,1993,NaN
4251,Europe,Serbia,SRB,RS,1994,NaN


In [81]:
# Observar datos Estonia
df_nulos[df_nulos["Country_normalizado"] == "Estonia"]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
3472,Europe,Estonia,EST,EE,1990,NaN
3473,Europe,Estonia,EST,EE,1991,NaN
3474,Europe,Estonia,EST,EE,1992,NaN


In [82]:
# Observar datos Yemen
df_nulos[df_nulos["Country_normalizado"] == "Yemen, Rep."]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
3067,Asia,"Yemen, Rep.",YEM,YE,2019,NaN
3068,Asia,"Yemen, Rep.",YEM,YE,2020,NaN


In [83]:
# Observar datos Mozambique
df_nulos[df_nulos["Country_normalizado"] == "Mozambique"]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
1054,Africa,Mozambique,MOZ,MZ,1990,NaN


- Se puede encontrar estos datos de manera manual?
- No estan porque no existen?
- Los que tienen pocos se pueden completar a partir de los datos alrededor?

### Chequear datos cercanos a los nulos

In [84]:
# Ver datos Mozambique
df_full_pbi[(df_full_pbi["Country_normalizado"] == "Mozambique") & (df_full_pbi["Años"].isin([1990, 1991, 1992, 1993, 1994]))]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
1054,Africa,Mozambique,MOZ,MZ,1990,NaN
1055,Africa,Mozambique,MOZ,MZ,1991,288.71
1056,Africa,Mozambique,MOZ,MZ,1992,205.59
1057,Africa,Mozambique,MOZ,MZ,1993,205.19
1058,Africa,Mozambique,MOZ,MZ,1994,196.76


In [85]:
# Ver datos Yemen
df_full_pbi[(df_full_pbi["Country_normalizado"] == "Yemen, Rep.") & (df_full_pbi["Años"].isin([2014, 2015, 2016, 2017, 2018, 2019]))]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
3062,Asia,"Yemen, Rep.",YEM,YE,2014,1430.16
3063,Asia,"Yemen, Rep.",YEM,YE,2015,1362.17
3064,Asia,"Yemen, Rep.",YEM,YE,2016,975.36
3065,Asia,"Yemen, Rep.",YEM,YE,2017,811.17
3066,Asia,"Yemen, Rep.",YEM,YE,2018,633.89
3067,Asia,"Yemen, Rep.",YEM,YE,2019,NaN


In [86]:
# Ver datos Venezuela
df_full_pbi[(df_full_pbi["Country_normalizado"] == "Venezuela, Rb") & (df_full_pbi["Años"].isin([2010, 2011, 2012, 2013, 2014, 2015]))]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
6003,South America,"Venezuela, Rb",VEN,VE,2010,13646.30
6004,South America,"Venezuela, Rb",VEN,VE,2011,10843.92
6005,South America,"Venezuela, Rb",VEN,VE,2012,12901.42
6006,South America,"Venezuela, Rb",VEN,VE,2013,12403.15
6007,South America,"Venezuela, Rb",VEN,VE,2014,15943.61
6008,South America,"Venezuela, Rb",VEN,VE,2015,NaN


In [87]:
# Ver datos Latvia
df_full_pbi[(df_full_pbi["Country_normalizado"] == "Latvia") & (df_full_pbi["Años"].isin([1994, 1995, 1996, 1997, 1998, 1999]))]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
3786,Europe,Latvia,LVA,LV,1994,NaN
3787,Europe,Latvia,LVA,LV,1995,2256.77
3788,Europe,Latvia,LVA,LV,1996,2360.17
3789,Europe,Latvia,LVA,LV,1997,2609.89
3790,Europe,Latvia,LVA,LV,1998,2893.80
3791,Europe,Latvia,LVA,LV,1999,3063.90


In [88]:
# Ver datos Lithuania
df_full_pbi[(df_full_pbi["Country_normalizado"] == "Lithuania") & (df_full_pbi["Años"].isin([1994, 1995, 1996, 1997, 1998, 1999]))]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
3848,Europe,Lithuania,LTU,LT,1994,NaN
3849,Europe,Lithuania,LTU,LT,1995,2182.69
3850,Europe,Lithuania,LTU,LT,1996,2340.68
3851,Europe,Lithuania,LTU,LT,1997,2844.16
3852,Europe,Lithuania,LTU,LT,1998,3180.64
3853,Europe,Lithuania,LTU,LT,1999,3127.51


In [89]:
# Ver datos Serbia
df_full_pbi[(df_full_pbi["Country_normalizado"] == "Serbia") & (df_full_pbi["Años"].isin([1994, 1995, 1996, 1997, 1998, 1999]))]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
4251,Europe,Serbia,SRB,RS,1994,NaN
4252,Europe,Serbia,SRB,RS,1995,2349.41
4253,Europe,Serbia,SRB,RS,1996,3053.91
4254,Europe,Serbia,SRB,RS,1997,3573.50
4255,Europe,Serbia,SRB,RS,1998,2775.47
4256,Europe,Serbia,SRB,RS,1999,2768.91


In [90]:
# Ver datos Estonia
df_full_pbi[(df_full_pbi["Country_normalizado"] == "Estonia") & (df_full_pbi["Años"].isin([1992, 1993, 1994, 1995, 1996, 1997]))]

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD
3474,Europe,Estonia,EST,EE,1992,NaN
3475,Europe,Estonia,EST,EE,1993,2685.91
3476,Europe,Estonia,EST,EE,1994,2819.13
3477,Europe,Estonia,EST,EE,1995,3134.39
3478,Europe,Estonia,EST,EE,1996,3380.93
3479,Europe,Estonia,EST,EE,1997,3682.95


- Se observan variaciones año a año en todos los paises
- Creo que para el proposito de este trabajo rellenar los nulos restantes con .ffill y .bfill sería suficiente y no modificaría sustancialmente el resultado del análisis.

### Relleno con .bfill y .ffill

In [91]:
# Cambios en el df ya sin paises con muchos nulos
df_full_pbi_clean = (
    df_full_pbi_clean
    # Asegurarnos que este ordenado por pais y luego por año
    .sort_values(["Country_normalizado", "Años"])
    # Tener en cuenta que no se rellene con datos de otro pais
    .groupby("Country_normalizado", group_keys=False)
    .apply(lambda g: g.assign(
        PIB_per_capita_USD=g["PIB_per_capita_USD"]
        .bfill()  # rellena los NaN iniciales con el primer valor disponible
        .ffill()  # rellena los NaN finales con el último valor disponible
    ))
)

/tmp/ipython-input-1237816533.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.assign(


In [92]:
# Chequear nulos
df_full_pbi_clean.isnull().sum()

,0
Continent,0
Country_normalizado,0
3_letter_code,0
2_letter_code,0
Años,0
PIB_per_capita_USD,0


In [93]:
# Shape
df_full_pbi_clean.shape

(5797, 6)

# Conección con datos educación superior

## API World Bank

In [94]:
# Transformar codigos de dos letras en lista para hacer un loop
paises = df_full_pbi["2_letter_code"].dropna().unique().tolist()
# Transformar lista de años en rango
años = list(range(1990, 2021))
# Indicador para acceder al PBI de cada país
indicador_edu = "SE.TER.ENRR"
# Definir lista para guardar resultados
rows_ed = []
# Calcular total de paises para establerer un porcentaje de progreso
total = len(paises)

# Loop
for i, codigo in enumerate(paises, 1):  # enumerate para saber en qué país vas
    # Url conección, necesita el codigo del páis, el indicador de la información que se busca y el rango de años
    url = f"https://api.worldbank.org/v2/country/{codigo}/indicator/{indicador_edu}?date=1990:2020&format=json&per_page=1000"
    response = requests.get(url)
    # Asegurarse una conección estable
    if response.status_code == 200:
        # Almacenamiento de los datos en lenguaje json
        data = response.json()
        # Si hay datos
        if len(data) > 1:
            # Por entrada de datos
            for entry in data[1]:
                # a veces la API devuelve valores nulos explícitamente
                # Append datos
                rows_ed.append({
                    "codigo_pais": codigo,
                    "año": int(entry["date"]),
                    "valor": entry["value"]
                })
    else:
        print(f"Error en {codigo}: {response.status_code}")

    # Mostrar progreso (cada 10 países aprox.)
    if i % 10 == 0 or i == total:
        progreso = (i / total) * 100
        print(f"Progreso: {i}/{total} países ({progreso:.1f}%) completado")

    time.sleep(0.5)

Progreso: 10/194 países (5.2%) completado
Progreso: 20/194 países (10.3%) completado
Progreso: 30/194 países (15.5%) completado
Progreso: 40/194 países (20.6%) completado
Progreso: 50/194 países (25.8%) completado
Progreso: 60/194 países (30.9%) completado
Progreso: 70/194 países (36.1%) completado
Progreso: 80/194 países (41.2%) completado
Progreso: 90/194 países (46.4%) completado
Progreso: 100/194 países (51.5%) completado
Progreso: 110/194 países (56.7%) completado
Progreso: 120/194 países (61.9%) completado
Progreso: 130/194 países (67.0%) completado
Progreso: 140/194 países (72.2%) completado
Progreso: 150/194 países (77.3%) completado
Progreso: 160/194 países (82.5%) completado
Progreso: 170/194 países (87.6%) completado
Progreso: 180/194 países (92.8%) completado
Progreso: 190/194 países (97.9%) completado
Progreso: 194/194 países (100.0%) completado


In [95]:
# Pasar a dataframe
df_edu = pd.DataFrame(rows_ed)
df_edu.head()

,codigo_pais,año,valor
0,DZ,2020,53.375219
1,DZ,2019,53.864502
2,DZ,2018,52.881939
3,DZ,2017,49.177690
4,DZ,2016,43.904062


In [96]:
# shape
df_edu.shape

(6014, 3)

In [97]:
# Chequear Nulos
df_edu.isnull().sum()

,0
codigo_pais,0
año,0
valor,2878


In [98]:
# Convertir en .csv
df_edu.to_csv("educacion_superior.csv", index=False)

In [99]:
# Cobertura para cada país:
df_coverage = (
    df_edu
    .groupby("codigo_pais")["valor"]
    .apply(lambda s: s.notna().sum() / len(años))
    .reset_index()
    .rename(columns={"valor": "fraccion_cobertura"})
)

print(df_coverage.sort_values("fraccion_cobertura", ascending=False))

    codigo_pais  fraccion_cobertura
26           BY                 1.0
5            AM                 1.0
39           CU                 1.0
36           CN                 1.0
88           KG                 1.0
..          ...                 ...
133          NR                 0.0
161          SO                 0.0
151          SB                 0.0
163          SS                 0.0
178          TV                 0.0

[194 rows x 2 columns]


In [100]:
# cuantos paises tienen 0.0
df_coverage[df_coverage["fraccion_cobertura"] == 0.0]

,codigo_pais,fraccion_cobertura
21,BO,0.0
74,HT,0.0
90,KI,0.0
133,NR,0.0
151,SB,0.0
161,SO,0.0
163,SS,0.0
178,TV,0.0


In [101]:
# Cuantos paises tienen la mitad o menos
df_coverage[df_coverage["fraccion_cobertura"] <= 0.5]

,codigo_pais,fraccion_cobertura
0,AD,0.483871
1,AE,0.096774
2,AF,0.258065
3,AG,0.129032
6,AO,0.451613
...,...,...
189,WS,0.225806
190,YE,0.419355
191,ZA,0.451613
192,ZM,0.161290


In [102]:
# Observar ZA en df_edu
df_edu[df_edu["codigo_pais"] == "ZA"]

,codigo_pais,año,valor
1364,ZA,2020,22.614874
1365,ZA,2019,22.244247
1366,ZA,2018,22.351543
1367,ZA,2017,21.090291
1368,ZA,2016,18.929298
1369,ZA,2015,19.033295
1370,ZA,2014,18.481336
1371,ZA,2013,18.608823
1372,ZA,2012,17.944723
1373,ZA,2011,NaN


In [103]:
# Paises con más de la mitad
df_coverage[df_coverage["fraccion_cobertura"] > 0.5]

,codigo_pais,fraccion_cobertura
4,AL,0.612903
5,AM,1.000000
7,AR,0.806452
11,BA,0.677419
12,BB,0.645161
...,...,...
180,UA,0.677419
181,UG,0.645161
183,UY,0.677419
184,UZ,0.677419


In [104]:
# chequear AR en df_edu
df_edu[df_edu["codigo_pais"] == "AR"]

,codigo_pais,año,valor
5642,AR,2020,99.369735
5643,AR,2019,95.192824
5644,AR,2018,91.084651
5645,AR,2017,89.329735
5646,AR,2016,86.674383
5647,AR,2015,83.755693
5648,AR,2014,81.072590
5649,AR,2013,78.568165
5650,AR,2012,78.146517
5651,AR,2011,77.010119


In [105]:
# Cuantos tienen completo
df_coverage[df_coverage["fraccion_cobertura"] == 1.0]

,codigo_pais,fraccion_cobertura
5,AM,1.0
26,BY,1.0
36,CN,1.0
39,CU,1.0
76,ID,1.0
88,KG,1.0
117,MN,1.0
150,SA,1.0
174,TN,1.0


In [106]:
# Redondear a .1
df_coverage["fraccion_cobertura"] = df_coverage["fraccion_cobertura"].round(1)

# Coneccion con Unesco datos de educación **superior**

## Conección y acceso a indicadores y codigos de paises

In [107]:
# Instalación librería Unesco
!pip install unesco-reader

In [108]:
# Acceso a la librería
import unesco_reader as uis

In [109]:
# Buscar indicadores que contengan "tertiary"
indicadores = uis.available_indicators(theme="EDUCATION")
indicadores

,indicatorCode,name,theme,lastDataUpdate,lastDataUpdateDescription,min,max,totalRecordCount,geoUnitType,last_data_update
0,10,Official entrance age to early childhood educa...,EDUCATION,9/17/2025,September 2025 Data Release,1972,2024,1738,NATIONAL,2025-09-17
1,10403,Start month of the academic school year (terti...,EDUCATION,9/17/2025,September 2025 Data Release,1991,2024,5329,NATIONAL,2025-09-17
2,10404,End month of the academic school year (tertiar...,EDUCATION,9/17/2025,September 2025 Data Release,1991,2024,5328,NATIONAL,2025-09-17
3,10405,Start of the academic school year (tertiary ed...,EDUCATION,9/17/2025,September 2025 Data Release,1970,2024,11098,NATIONAL,2025-09-17
4,10406,End of the academic school year (tertiary educ...,EDUCATION,9/17/2025,September 2025 Data Release,1991,2024,5351,NATIONAL,2025-09-17
...,...,...,...,...,...,...,...,...,...,...
4554,YADULT.PROFINUMERACY.NPIA,Proportion of population achieving at least a ...,EDUCATION,9/17/2025,September 2025 Data Release,2012,2023,64,NATIONAL,2025-09-17
4555,YEARS.FC.COMP.02,Number of years of compulsory pre-primary educ...,EDUCATION,9/17/2025,September 2025 Data Release,1975,2024,5567,NATIONAL,2025-09-17
4556,YEARS.FC.COMP.1T3,Number of years of compulsory primary and seco...,EDUCATION,9/17/2025,September 2025 Data Release,1975,2024,5557,NATIONAL,2025-09-17
4557,YEARS.FC.FREE.02,Number of years of free pre-primary education ...,EDUCATION,9/17/2025,September 2025 Data Release,1998,2024,4543,NATIONAL,2025-09-17


In [110]:
# info
indicadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4559 entries, 0 to 4558
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   indicatorCode              4559 non-null   object        
 1   name                       4559 non-null   object        
 2   theme                      4559 non-null   object        
 3   lastDataUpdate             4559 non-null   object        
 4   lastDataUpdateDescription  4559 non-null   object        
 5   min                        4559 non-null   int64         
 6   max                        4559 non-null   int64         
 7   totalRecordCount           4559 non-null   int64         
 8   geoUnitType                4559 non-null   object        
 9   last_data_update           4559 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 356.3+ KB


In [111]:
indicadores["name"].head(10)

,name
0,Official entrance age to early childhood educa...
1,Start month of the academic school year (terti...
2,End month of the academic school year (tertiar...
3,Start of the academic school year (tertiary ed...
4,End of the academic school year (tertiary educ...
5,Theoretical duration of early childhood educat...
6,"Enrolment in pre-primary education, both sexes..."
7,"Enrolment in pre-primary education, female (nu..."
8,"Enrolment in primary education, both sexes (nu..."
9,"Enrolment in primary education, female (number)"


In [112]:
# Encontrar la matriculación a educación superior
# Filtrar por indicadores que contengan "tertiary" y "enrolment"
mask = ( indicadores["name"].str.contains("enrol", case=False, na=False) &
        indicadores["name"].str.contains("tertiary", case=False, na=False) &
         indicadores["name"].str.contains("percent", case=False, na=False))

indicadores_tertiary = indicadores[mask]

# Shape
indicadores_tertiary.shape

(16, 10)

In [113]:
# Mostrar resultados
indicadores_tertiary[["indicatorCode", "name"]].head(16)

,indicatorCode,name
1717,ETOIP.4.PR,Percentage of enrolment in post-secondary non-...
1718,ETOIP.4.PR.F,Percentage of enrolment in post-secondary non-...
1719,ETOIP.4.PR.GPIA,Percentage of enrolment in post-secondary non-...
1720,ETOIP.4.PR.M,Percentage of enrolment in post-secondary non-...
1721,ETOIP.4.PU,Percentage of enrolment in post-secondary non-...
1722,ETOIP.4.PU.F,Percentage of enrolment in post-secondary non-...
1723,ETOIP.4.PU.GPIA,Percentage of enrolment in post-secondary non-...
1724,ETOIP.4.PU.M,Percentage of enrolment in post-secondary non-...
1725,ETOIP.5T8.PR,Percentage of enrolment in tertiary education ...
1726,ETOIP.5T8.PR.F,Percentage of enrolment in tertiary education ...


In [114]:
# Chequear codigos en Unesco
geo_units = uis.available_geo_units()
geo_units_national = geo_units[geo_units['type'] == 'NATIONAL']
geo_units_national.head()

,id,name,type,regionGroup
0,ABW,Aruba,NATIONAL,NaN
1,AFG,Afghanistan,NATIONAL,NaN
2,AGO,Angola,NATIONAL,NaN
3,AIA,Anguilla,NATIONAL,NaN
4,ALA,Åland Islands,NATIONAL,NaN


In [115]:
print(f"Total de países reconocidos por UNESCO: {len(geo_units_national)}")

Total de países reconocidos por UNESCO: 241


### Chequeo de coincidencia en codigos de 3 letras

In [116]:
# Comparar con dataframe principal
col_codigo = "3_letter_code"

# Verificar si existen códigos que no están en UNESCO
codigos_fuera_unesco = df_full_pbi_clean.loc[
    ~df_full_pbi_clean[col_codigo].isin(geo_units_national["id"])
]

# Mostrar resultados
if codigos_fuera_unesco.empty:
    print("✅ Todos los países del dataset coinciden con los códigos UNESCO.")
else:
    print("⚠️ Algunos países no tienen correspondencia en UNESCO:")
    print(codigos_fuera_unesco[[col_codigo, "Country"]].drop_duplicates())
    print(f"\nTotal sin correspondencia: {codigos_fuera_unesco[col_codigo].nunique()}")

✅ Todos los países del dataset coinciden con los códigos UNESCO.


## Descarga de datos

In [117]:
# Indicador UNESCO para tertiary education (% enrolment)
indicador_terciario = "ETOIP.5T8.PR"

# Lista de países (pueden ser todos o los de tu dataset)
paises = df_full_pbi_clean["3_letter_code"].dropna().unique().tolist()

# Años de interés
start_year = 1990
end_year = 2022

# Total paises
total = len(paises)

# Lista para guardar resultados
rows_uis = []

print(f"Descargando datos de UNESCO ({indicador_terciario}) para {len(paises)} países...")

for i, codigo_iso3 in enumerate(paises, 1):
    try:
        # Descargar todo el rango para cada país
        df_temp = uis.get_data(
            indicador_terciario,
            codigo_iso3,
            start=start_year,
            end=end_year,
            footnotes=False,
            labels=True  # labels=True para tener nombres de columnas legibles
        )

        if df_temp is not None and not df_temp.empty:
            df_temp["codigo_pais"] = codigo_iso3
            rows_uis.append(df_temp)
            print(f"✅ {codigo_iso3}: {len(df_temp)} filas")
        else:
            print(f"⚠️ {codigo_iso3}: sin datos")

    except Exception as e:
        print(f"❌ Error con {codigo_iso3}: {e}")

    # Evita saturar el servidor
    time.sleep(0.5)

    # Mostrar progreso cada 10 países
    if i % 10 == 0 or i == total:
        print(f"Progreso: {i}/{len(paises)} países completados")

# Combinar todo en un DataFrame
if rows_uis:
    df_uis = pd.concat(rows_uis, ignore_index=True)
    print(f"\nTotal de filas descargadas: {len(df_uis)}")
else:
    df_uis = pd.DataFrame()
    print("\nNo se descargaron datos.")

df_uis.head()

Descargando datos de UNESCO (ETOIP.5T8.PR) para 187 países...
✅ ALB: 15 filas
✅ DZA: 11 filas
✅ AND: 10 filas
✅ AGO: 8 filas
✅ ATG: 4 filas
✅ ARG: 22 filas
✅ ARM: 10 filas
✅ AUS: 8 filas
✅ AUT: 16 filas
✅ AZE: 17 filas
Progreso: 10/187 países completados
❌ Error con BHS: No data found for the given parameters
✅ BHR: 9 filas
✅ BGD: 20 filas
❌ Error con BRB: No data found for the given parameters
✅ BLR: 23 filas
✅ BEL: 16 filas
✅ BLZ: 12 filas
✅ BEN: 17 filas
✅ BTN: 3 filas
❌ Error con BOL: No data found for the given parameters
Progreso: 20/187 países completados
✅ BIH: 13 filas
✅ BWA: 12 filas
✅ BRA: 23 filas
✅ BRN: 15 filas
✅ BGR: 24 filas
✅ BFA: 18 filas
✅ BDI: 11 filas
✅ CPV: 15 filas
✅ KHM: 11 filas
✅ CMR: 19 filas
Progreso: 30/187 países completados
❌ Error con CAN: No data found for the given parameters
✅ CAF: 3 filas
✅ TCD: 7 filas
✅ CHL: 22 filas
✅ CHN: 10 filas
✅ COL: 23 filas
❌ Error con COM: No data found for the given parameters
✅ COD: 3 filas
✅ COG: 10 filas
✅ CRI: 9 filas

,indicatorId,geoUnit,year,value,magnitude,qualifier,name,geoUnitName,regionGroup,codigo_pais
0,ETOIP.5T8.PR,ALB,2003,0.00000,NIL,None,Percentage of enrolment in tertiary education ...,Albania,None,ALB
1,ETOIP.5T8.PR,ALB,2004,0.76395,None,None,Percentage of enrolment in tertiary education ...,Albania,None,ALB
2,ETOIP.5T8.PR,ALB,2005,1.55398,None,None,Percentage of enrolment in tertiary education ...,Albania,None,ALB
3,ETOIP.5T8.PR,ALB,2006,2.29574,None,None,Percentage of enrolment in tertiary education ...,Albania,None,ALB
4,ETOIP.5T8.PR,ALB,2007,6.63228,None,None,Percentage of enrolment in tertiary education ...,Albania,None,ALB


In [118]:
# Drop columnas dejar codigo_pais, year y value
df_uis_simple = df_uis[["codigo_pais", "year", "value"]]
df_uis_simple.head()

,codigo_pais,year,value
0,ALB,2003,0.00000
1,ALB,2004,0.76395
2,ALB,2005,1.55398
3,ALB,2006,2.29574
4,ALB,2007,6.63228


# Union con df_edu

In [119]:
# Dejar df_full_pbi sin paises duplicados
df_codigos = df_full_pbi[["2_letter_code", "3_letter_code"]].drop_duplicates()

# agregar codigo de 3 letras a df_edu
df_edu2 = pd.merge(df_edu, df_codigos[["2_letter_code", "3_letter_code"]], left_on="codigo_pais", right_on="2_letter_code", how="left")
df_edu2.head()


,codigo_pais,año,valor,2_letter_code,3_letter_code
0,DZ,2020,53.375219,DZ,DZA
1,DZ,2019,53.864502,DZ,DZA
2,DZ,2018,52.881939,DZ,DZA
3,DZ,2017,49.177690,DZ,DZA
4,DZ,2016,43.904062,DZ,DZA


In [120]:
# Merge df_edu2 y df_uis_simple
df_edu_uis = pd.merge(df_edu2, df_uis_simple, left_on=["3_letter_code", "año"], right_on=["codigo_pais", "year"], how="left")
df_edu_uis.head()

,codigo_pais_x,año,valor,2_letter_code,3_letter_code,codigo_pais_y,year,value
0,DZ,2020,53.375219,DZ,DZA,DZA,2020.0,0.0
1,DZ,2019,53.864502,DZ,DZA,DZA,2019.0,0.0
2,DZ,2018,52.881939,DZ,DZA,DZA,2018.0,0.0
3,DZ,2017,49.177690,DZ,DZA,DZA,2017.0,0.0
4,DZ,2016,43.904062,DZ,DZA,DZA,2016.0,0.0


In [121]:
# shape
df_edu_uis.shape

(6014, 8)

In [122]:
# Ordenar alfabeticamente por codigo_pais y despues por año
df_edu_uis = df_edu_uis.sort_values(["3_letter_code", "año"])
df_edu_uis.head()

,codigo_pais_x,año,valor,2_letter_code,3_letter_code,codigo_pais_y,year,value
1704,AF,1990,2.2152,AF,AFG,NaN,NaN,NaN
1703,AF,1991,NaN,AF,AFG,NaN,NaN,NaN
1702,AF,1992,NaN,AF,AFG,NaN,NaN,NaN
1701,AF,1993,NaN,AF,AFG,NaN,NaN,NaN
1700,AF,1994,NaN,AF,AFG,NaN,NaN,NaN


In [123]:
# rows por 3_letter_code
df_edu_uis["3_letter_code"].value_counts()

,count
3_letter_code,
AFG,31
AGO,31
ALB,31
AND,31
ARE,31
...,...
WSM,31
YEM,31
ZAF,31


In [124]:
# chequeo nulos
df_edu_uis.isnull().sum()

,0
codigo_pais_x,0
año,0
valor,2878
2_letter_code,0
3_letter_code,0
codigo_pais_y,4078
year,4078
value,4078


## Trabajo Nulos

In [125]:
# Chequear si algun nulo valor no es nulo en value
df_edu_uis[df_edu_uis["valor"].isnull() & df_edu_uis["value"].notnull()]

,codigo_pais_x,año,valor,2_letter_code,3_letter_code,codigo_pais_y,year,value
3338,BG,1999,NaN,BG,BGR,BGR,1999.0,12.031380
5726,BR,1998,NaN,BR,BRA,BRA,1998.0,60.566959
5725,BR,1999,NaN,BR,BRA,BRA,1999.0,63.055820
5724,BR,2000,NaN,BR,BRA,BRA,2000.0,65.420982
5723,BR,2001,NaN,BR,BRA,BRA,2001.0,67.361031
...,...,...,...,...,...,...,...,...
2873,TH,1999,NaN,TH,THA,THA,1999.0,21.030420
2872,TH,2000,NaN,TH,THA,THA,2000.0,19.538830
5570,TO,1999,NaN,TO,TON,TON,1999.0,67.032967
5973,UY,1999,NaN,UY,URY,URY,1999.0,10.898010


In [126]:
# Combinar columnas valor y value, cuando valor es nula y value no, usar value
df_edu_uis["valor"] = df_edu_uis.apply(lambda row: row["value"] if pd.isnull(row["valor"]) else row["valor"], axis=1)

In [127]:
# Check nulos
df_edu_uis.isnull().sum()

,0
codigo_pais_x,0
año,0
valor,2810
2_letter_code,0
3_letter_code,0
codigo_pais_y,4078
year,4078
value,4078


In [128]:
# Dejar solo 2_letter_code, año, valor
df_edu_uis2 = df_edu_uis[["2_letter_code", "año", "valor"]]
df_edu_uis2.head()


,2_letter_code,año,valor
1704,AF,1990,2.2152
1703,AF,1991,NaN
1702,AF,1992,NaN
1701,AF,1993,NaN
1700,AF,1994,NaN


In [129]:
# shape
df_edu_uis2.shape

(6014, 3)

### Completar Nulos

Teniendo en cuenta el proposito del trabajo voy a completar los nulos para contar con un dataset que permita el posterior trabajo de machine learning, sin priorizar que los mismos sean un reflejo adecuado de la realidad.

In [130]:
# Ordenar por país y año
df_edu_uis2 = df_edu_uis2.sort_values(["2_letter_code", "año"])

# Interpolación lineal por grupo (país)
df_edu_uis2["valor_filled"] = df_edu_uis2.groupby("2_letter_code")["valor"].transform(lambda g: g.interpolate(method='linear'))

# Si hay NaN al inicio o final (sin valor anterior/siguiente), rellenar con el primer/último disponible
df_edu_uis2["valor_filled"] = df_edu_uis2.groupby("2_letter_code")["valor_filled"].transform(lambda g: g.fillna(method='bfill').fillna(method='ffill'))

df_edu_uis2.head()

/tmp/ipython-input-1082642919.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_edu_uis2["valor_filled"] = df_edu_uis2.groupby("2_letter_code")["valor_filled"].transform(lambda g: g.fillna(method='bfill').fillna(method='ffill'))


,2_letter_code,año,valor,valor_filled
3130,AD,1990,NaN,7.30906
3129,AD,1991,NaN,7.30906
3128,AD,1992,NaN,7.30906
3127,AD,1993,NaN,7.30906
3126,AD,1994,NaN,7.30906


In [131]:
# shape
df_edu_uis2.shape

(6014, 4)

In [132]:
# Nulos
df_edu_uis2.isnull().sum()

,0
2_letter_code,0
año,0
valor,2810
valor_filled,248


In [133]:
# 248 Nulos restantes, chequear si son paises sin ningun dato
# Ver cuántos valores nulos por país
nulos_por_pais = df_edu_uis2.groupby("2_letter_code")["valor_filled"].apply(lambda x: x.isnull().all())

# Filtrar solo los países donde todos los valores son NaN
paises_sin_datos = nulos_por_pais[nulos_por_pais].index.tolist()

print(f"Países sin ningún dato: {len(paises_sin_datos)}")
print(paises_sin_datos)

Países sin ningún dato: 8
['BO', 'HT', 'KI', 'NR', 'SB', 'SO', 'SS', 'TV']


In [134]:
# Eliminar paises sin datos

# Filtrar el DataFrame para excluir esos países
df_edu_uis_clean = df_edu_uis2[~df_edu_uis["2_letter_code"].isin(paises_sin_datos)].copy()

# Verificar
print(df_edu_uis_clean["2_letter_code"].nunique(), "países restantes")
print(df_edu_uis_clean.isnull().sum())

186 países restantes
2_letter_code       0
año                 0
valor            2562
valor_filled        0
dtype: int64


/tmp/ipython-input-3795668768.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_edu_uis_clean = df_edu_uis2[~df_edu_uis["2_letter_code"].isin(paises_sin_datos)].copy()


In [135]:
# Sacar la columna valor
df_edu_uis_clean = df_edu_uis_clean[["2_letter_code", "año", "valor_filled"]]

# Redondear valor_filled
df_edu_uis_clean["valor_filled"] = df_edu_uis_clean["valor_filled"].round(1)

df_edu_uis_clean.head()

,2_letter_code,año,valor_filled
3130,AD,1990,7.3
3129,AD,1991,7.3
3128,AD,1992,7.3
3127,AD,1993,7.3
3126,AD,1994,7.3


# Merge con df principal

In [136]:
# Unir con df_pbi_full_clean
df_full_matric = pd.merge(
    df_full_pbi_clean,
    df_edu_uis_clean,
    left_on=["2_letter_code", "Años"],
    right_on=["2_letter_code", "año"]
)
df_full_matric.head()

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD,año,valor_filled
0,Europe,Albania,ALB,AL,1990,617.23,1990,16.4
1,Europe,Albania,ALB,AL,1991,336.59,1991,16.4
2,Europe,Albania,ALB,AL,1992,200.85,1992,16.4
3,Europe,Albania,ALB,AL,1993,367.28,1993,16.4
4,Europe,Albania,ALB,AL,1994,586.42,1994,16.4


In [137]:
# shape
df_full_matric.shape

(5580, 8)

In [138]:
# Datos por pais
df_full_matric["2_letter_code"].value_counts()

,count
2_letter_code,
AL,31
DZ,31
AD,31
AO,31
AG,31
...,...
VE,31
VN,31
YE,31


In [139]:
# Cantidad de paises
df_full_matric["2_letter_code"].nunique()

180

In [140]:
# Chequeo nulos
df_full_matric.isnull().sum()

,0
Continent,0
Country_normalizado,0
3_letter_code,0
2_letter_code,0
Años,0
PIB_per_capita_USD,0
año,0
valor_filled,0


In [141]:
# guardar en archivo .csv
df_full_matric.to_csv("df_full_matric.csv", index=False)

# Natalidad

In [142]:
# Lista de países (códigos de 2 letras)
paises = df_full_pbi_clean["2_letter_code"].dropna().unique().tolist()

# Rango de años: 1971–2002 (19 años antes del periodo de matrícula)
años = list(range(1971, 2002))

# Indicador: tasa de natalidad
indicador_natalidad = "SP.DYN.CBRT.IN"

# Total paises
total = len(paises)

# Lista para guardar resultados
rows_natalidad = []

for i, codigo in enumerate(paises, 1):
    url = f"https://api.worldbank.org/v2/country/{codigo}/indicator/{indicador_natalidad}?date=1971:2002&format=json&per_page=1000"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if len(data) > 1:
            for entry in data[1]:
                if entry["value"] is not None:
                    rows_natalidad.append({
                        "codigo_pais": codigo,
                        "año": int(entry["date"]),
                        "tasa_natalidad": entry["value"]
                    })
    else:
        print(f"Error en {codigo}: {response.status_code}")

    if i % 10 == 0 or i == total:
        print(f"Progreso: {i}/{len(paises)} países completados")
    time.sleep(0.5)

df_natalidad = pd.DataFrame(rows_natalidad)

Progreso: 10/187 países completados
Progreso: 20/187 países completados
Progreso: 30/187 países completados
Progreso: 40/187 países completados
Progreso: 50/187 países completados
Progreso: 60/187 países completados
Progreso: 70/187 países completados
Progreso: 80/187 países completados
Progreso: 90/187 países completados
Progreso: 100/187 países completados
Progreso: 110/187 países completados
Progreso: 120/187 países completados
Progreso: 130/187 países completados
Progreso: 140/187 países completados
Progreso: 150/187 países completados
Progreso: 160/187 países completados
Progreso: 170/187 países completados
Progreso: 180/187 países completados
Progreso: 187/187 países completados


In [143]:
# Agregar año matricula para merge
df_natalidad["año_matricula"] = df_natalidad["año"] + 19
df_natalidad.head()

,codigo_pais,año,tasa_natalidad,año_matricula
0,AL,2002,15.585,2021
1,AL,2001,16.596,2020
2,AL,2000,17.513,2019
3,AL,1999,18.487,2018
4,AL,1998,19.609,2017


In [144]:
# info
df_natalidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5984 entries, 0 to 5983
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   codigo_pais     5984 non-null   object 
 1   año             5984 non-null   int64  
 2   tasa_natalidad  5984 non-null   float64
 3   año_matricula   5984 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 187.1+ KB


In [145]:
# shape
df_natalidad.shape

(5984, 4)

In [146]:
# nulos
df_natalidad.isnull().sum()

,0
codigo_pais,0
año,0
tasa_natalidad,0
año_matricula,0


In [147]:
# Cantidad de valores por codigo_pais
df_natalidad["codigo_pais"].value_counts()

,count
codigo_pais,
AL,32
DZ,32
AD,32
AO,32
AG,32
...,...
VE,32
VN,32
YE,32


## Merge

In [148]:
# Cargar df_full_matric.csv
df_full_matricula = pd.read_csv("df_full_matric.csv")
df_full_matricula

,Continent,Country_normalizado,3_letter_code,2_letter_code,Años,PIB_per_capita_USD,año,valor_filled
0,Europe,Albania,ALB,AL,1990,617.23,1990,16.4
1,Europe,Albania,ALB,AL,1991,336.59,1991,16.4
2,Europe,Albania,ALB,AL,1992,200.85,1992,16.4
3,Europe,Albania,ALB,AL,1993,367.28,1993,16.4
4,Europe,Albania,ALB,AL,1994,586.42,1994,16.4
...,...,...,...,...,...,...,...,...
5575,Africa,Zimbabwe,ZWE,ZW,2016,1407.42,2016,11.0
5576,Africa,Zimbabwe,ZWE,ZW,2017,3448.09,2017,11.0
5577,Africa,Zimbabwe,ZWE,ZW,2018,2271.85,2018,11.0
5578,Africa,Zimbabwe,ZWE,ZW,2019,1683.91,2019,11.0


In [149]:
# Eliminar 3_letter_code y año
df_full_matricula = df_full_matricula.drop(columns=["3_letter_code", "año"])
df_full_matricula.head()

,Continent,Country_normalizado,2_letter_code,Años,PIB_per_capita_USD,valor_filled
0,Europe,Albania,AL,1990,617.23,16.4
1,Europe,Albania,AL,1991,336.59,16.4
2,Europe,Albania,AL,1992,200.85,16.4
3,Europe,Albania,AL,1993,367.28,16.4
4,Europe,Albania,AL,1994,586.42,16.4


In [150]:
# Merge df_full_matricula con df_natalidad
df_full_matricula_natalidad = pd.merge(
    df_full_matricula, df_natalidad,
    left_on=["2_letter_code", "Años"],
    right_on=["codigo_pais", "año_matricula"],
    how="left"
)
df_full_matricula_natalidad.head()

,Continent,Country_normalizado,2_letter_code,Años,PIB_per_capita_USD,valor_filled,codigo_pais,año,tasa_natalidad,año_matricula
0,Europe,Albania,AL,1990,617.23,16.4,AL,1971,33.054,1990
1,Europe,Albania,AL,1991,336.59,16.4,AL,1972,32.623,1991
2,Europe,Albania,AL,1992,200.85,16.4,AL,1973,31.950,1992
3,Europe,Albania,AL,1993,367.28,16.4,AL,1974,31.452,1993
4,Europe,Albania,AL,1994,586.42,16.4,AL,1975,31.033,1994


In [151]:
# shape
df_full_matricula_natalidad.shape

(5580, 10)

# Arreglar DF Final

In [152]:
# Eliminar columnar 2_letter_code, codigo_pais, año_matricula
df_final = df_full_matricula_natalidad.drop(columns=["2_letter_code", "codigo_pais", "año_matricula"])
df_final.head()

,Continent,Country_normalizado,Años,PIB_per_capita_USD,valor_filled,año,tasa_natalidad
0,Europe,Albania,1990,617.23,16.4,1971,33.054
1,Europe,Albania,1991,336.59,16.4,1972,32.623
2,Europe,Albania,1992,200.85,16.4,1973,31.950
3,Europe,Albania,1993,367.28,16.4,1974,31.452
4,Europe,Albania,1994,586.42,16.4,1975,31.033


In [153]:
# Cambiar nombres columnas
df_final = df_final.rename(columns={
    "Continent": "Continente",
    "Country_normalizado": "Pais",
    "Años": "Año",
    "PBI_per_capita_USD": "PBI_PC_USD",
    "valor_filled": "Porcentaje_Matriculas",
    "año": "Año_Natalidad",
    "tasa_natalidad": "Tasa_Natalidad"
})
df_final.head()

,Continente,Pais,Año,PIB_per_capita_USD,Porcentaje_Matriculas,Año_Natalidad,Tasa_Natalidad
0,Europe,Albania,1990,617.23,16.4,1971,33.054
1,Europe,Albania,1991,336.59,16.4,1972,32.623
2,Europe,Albania,1992,200.85,16.4,1973,31.950
3,Europe,Albania,1993,367.28,16.4,1974,31.452
4,Europe,Albania,1994,586.42,16.4,1975,31.033


In [154]:
# Guardar en .csv
df_final.to_csv("df_final.csv", index=False)

## Explorar dataframe

In [155]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5580 entries, 0 to 5579
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Continente             5580 non-null   object 
 1   Pais                   5580 non-null   object 
 2   Año                    5580 non-null   int64  
 3   PIB_per_capita_USD     5580 non-null   float64
 4   Porcentaje_Matriculas  5580 non-null   float64
 5   Año_Natalidad          5580 non-null   int64  
 6   Tasa_Natalidad         5580 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 305.3+ KB


In [156]:
df_final.describe()

,Año,PIB_per_capita_USD,Porcentaje_Matriculas,Año_Natalidad,Tasa_Natalidad
count,5580.000000,5580.000000,5580.000000,5580.000000,5580.000000
mean,2005.000000,11540.208833,29.742115,1986.000000,29.707897
std,8.945073,20318.602873,27.350631,8.945073,12.942274
min,1990.000000,22.950000,0.000000,1971.000000,7.200000
25%,1997.000000,992.655000,7.300000,1978.000000,17.282750
50%,2005.000000,3489.525000,19.100000,1986.000000,29.550500
75%,2013.000000,12837.050000,50.500000,1994.000000,41.496000
max,2020.000000,204263.800000,144.000000,2001.000000,56.596000


In [157]:
df_final.shape

(5580, 7)

In [158]:
df_final.columns

Index(['Continente', 'Pais', 'Año', 'PIB_per_capita_USD',
       'Porcentaje_Matriculas', 'Año_Natalidad', 'Tasa_Natalidad'],
      dtype='object')

In [159]:
df_final.nunique()

,0
Continente,6
Pais,180
Año,31
PIB_per_capita_USD,5533
Porcentaje_Matriculas,908
Año_Natalidad,31
Tasa_Natalidad,4193


In [160]:
# Paises por continente
df_final.groupby("Continente")["Pais"].nunique()

,Pais
Continente,
Africa,50
Asia,42
Europe,45
North America,22
Oceania,10
South America,11


# Observaciones a partir de la exploración de datos

- Paises por continente: Todos los continentes tienen suficientes paises con datos para analizarlos, hay que tener en cuenta la diferencia en cantidad de paises en cada uno para un analisis correcto
- Se observan grandes diferencias entre el mínimo y máximo en el PBI per capita, el porcentaje de matriculas y la tasa de natalidad.


# Exploración gráfica

## Preguntas para guiar la exploración

- ¿Existe una correlación directa entre el PBI y la matriculación?
- ¿Existe una correlación directa entre la natalidad y la matriculación?
- ¿Qué factor tiene mayor influencia?
- ¿Hay continentes con una matriculación a la educación superior notablemente mayor que el resto?
- ¿Se encuentran los paises con mayor matriculación en el mismo continente?

## ¿Hay continentes con una marticulación notablemente mayor que el resto?

In [161]:
# Histograma continentes plotly
fig = px.histogram(
    df_final,
    x="Continente",
    title="Distribución de continentes",
    labels={"Continente": "Continente"},
    color_discrete_sequence=["blue"]
)
fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.show()

In [164]:
# Cantidad de paises por continente (gráfico de barras, plotly)
fig = px.bar(
    df_final.groupby("Continente")["Pais"].nunique().reset_index(),
    x="Continente",
    y="Pais",
    title="Cantidad de países por continente",
)
fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.show()


In [178]:
# Relacion entre paises y pbi per capita, top 10
# Guardar el pbi mayor de cada pais y el continente
# Obtener índice del valor máximo de PIB_per_capita_USD por país
idx = df_final.groupby("Pais")["PIB_per_capita_USD"].idxmax()

# Filtrar esas filas del DataFrame original
df_pbi_max = df_final.loc[idx, ["Pais", "Continente", "Año", "PIB_per_capita_USD"]].reset_index(drop=True)

# Gráfico
fig = px.scatter(
    df_pbi_max.sort_values("PIB_per_capita_USD", ascending=False).head(10),
    x="PIB_per_capita_USD",
    y="Pais",
    title="Top 8 países con mayor PBI per capita",
    labels={"PIB_per_capita_USD": "PBI per capita (USD)", "Pais": "País"},
    color_discrete_sequence=["blue"]
)
fig.show()

In [173]:
df_pbi_max.head()

,Pais,Continente,Año,PIB_per_capita_USD
0,Albania,Europe,2019,5460.43
1,Algeria,Africa,2014,6094.69
2,Andorra,Europe,2008,49132.52
3,Angola,Africa,2012,5086.03
4,Antigua And Barbuda,North America,2019,18896.37


In [176]:
# Agrupar df_pbi_max por continentes
fig = px.bar(
    df_pbi_max.groupby("Continente")["PIB_per_capita_USD"].sum().reset_index(),
    x="Continente",
    y="PIB_per_capita_USD",
    title="Total PBI_per_capita por Continente",
)
fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.show()

In [177]:
# Promedio de PBI_per_capita por continente
fig = px.bar(
    df_final.groupby("Continente")["PIB_per_capita_USD"].mean().reset_index(),
    x="Continente",
    y="PIB_per_capita_USD",
    title="Promedio de PBI per capita por continente",
)
fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.show()

In [286]:
# Mapa Matriculas por pais
fig = px.choropleth(
    df_final,
    locations="Pais",
    locationmode="country names",
    color="Porcentaje_Matriculas",
    hover_name="Pais",
    animation_frame="Año",
    color_continuous_scale="Purples",
    title="Porcentaje de Matrículas por País"
)

fig.update_layout(
    geo=dict(showframe=False, showcoastlines=False),
    coloraxis_colorbar=dict(title="Porcentaje de Matrículas (%)")
)

fig.show()

In [288]:
fig = px.scatter_geo(
    df_final,
    locations="Pais",
    locationmode="country names",
    color="Porcentaje_Matriculas",
    size="Porcentaje_Matriculas",
    hover_name="Pais",
    animation_frame="Año",
    projection="natural earth",
    color_continuous_scale="Viridis",
    title="Porcentaje de Matrículas por País (Scatter Geo)"
)

fig.update_layout(
    geo=dict(showframe=False, showcoastlines=True),
    coloraxis_colorbar=dict(title="Porcentaje Matrículas (%)")
)

fig.show()

Insights:

- 7 de los 10 paises con los mayores PBI_per_capita se encuentran en Europa
-  El Total del PBI per capita de los paises en Europa supera por casi el triple al continente que le sigue, incluso cuando no es el país con la mayor cantidad de paises.
- Europa tambien supera, por caso el triple, al continente que le sigue en el promedio del PBI per capita.

## ¿Se encuentran los paises con mayores matriculación en el mismo continente?

In [180]:
# Relacion entre matricula y páises, top 10
# Guardar el pbi mayor de cada pais y el continente
# Obtener índice del valor máximo de PIB_per_capita_USD por país
idx2 = df_final.groupby("Pais")["Porcentaje_Matriculas"].idxmax()

# Filtrar esas filas del DataFrame original
df_mtr_max = df_final.loc[idx2, ["Pais", "Continente", "Año", "Porcentaje_Matriculas"]].reset_index(drop=True)

# Gráfico
fig = px.scatter(
    df_mtr_max.sort_values("Porcentaje_Matriculas", ascending=False).head(10),
    x="Porcentaje_Matriculas",
    y="Pais",
    title="Top 10 países con mayor matriculas",
    labels={"Porcentaje_Matriculas": "Porcentaje Matriculas", "Pais": "País"},
    color_discrete_sequence=["blue"]
)
fig.show()

In [181]:
# Agrupar df_mtr_max por continente
fig = px.bar(
    df_mtr_max.groupby("Continente")["Porcentaje_Matriculas"].sum().reset_index(),
    x="Continente",
    y="Porcentaje_Matriculas",
    title="Total matriculas por Continente",
)
fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.show()

In [185]:
# Promedio Porcentaje_Matriculas por Continente
fig = px.pie(
    df_final.groupby("Continente")["Porcentaje_Matriculas"].mean().reset_index(),
    names="Continente",
    values="Porcentaje_Matriculas",
    title="Promedio de matriculas por continente",
)
fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.show()

Insights:

- A diferencia del PBI, se oberva mayor diversidad en los continentes de los paises que cuentan con los mayores porcentajes de matriculas.
- Europa muestra una gran superioridad en la totalidad de los porcentajes de matriculación
- Al observar el promedio en cada continente volvemos a encontrar una mayor paridad, representando Europa un cuarto del total.

## ¿Existe una correlación directa entre el PBI y la Matriculación?

In [161]:
# Grafico Scatter
# Asia matriculas y pbi en los años
fig = px.scatter(
    df_final[df_final['Continente'] == "Asia"],
    x="PIB_per_capita_USD",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre PBI per capita y matriculas en Asia",
    labels={"PIB_per_capita_USD": "PBI per capita (USD)", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [162]:
# Grafico Scatter
# Africa matriculas y pbi en los años
fig = px.scatter(
    df_final[df_final['Continente'] == "Africa"],
    x="PIB_per_capita_USD",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre PBI per capita y matriculas en Africa",
    labels={"PIB_per_capita_USD": "PBI per capita (USD)", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [163]:
# Grafico Scatter
# NorteAmerica matriculas y pbi en los años
fig = px.scatter(
    df_final[df_final['Continente'] == "North America"],
    x="PIB_per_capita_USD",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre PBI per capita y matriculas en America del Norte",
    labels={"PIB_per_capita_USD": "PBI per capita (USD)", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [164]:
# Grafico Scatter
# SurAmerica matriculas y pbi en los años
fig = px.scatter(
    df_final[df_final['Continente'] == "South America"],
    x="PIB_per_capita_USD",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre PBI per capita y matriculas en America del Sur",
    labels={"PIB_per_capita_USD": "PBI per capita (USD)", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [165]:
# Grafico Scatter
# Oceania matriculas y pbi en los años
fig = px.scatter(
    df_final[df_final['Continente'] == "Oceania"],
    x="PIB_per_capita_USD",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre PBI per capita y matriculas en Oceania",
    labels={"PIB_per_capita_USD": "PBI per capita (USD)", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [166]:
# Grafico Scatter
# Europa matriculas y pbi en los años
fig = px.scatter(
    df_final[df_final['Continente'] == "Europe"],
    x="PIB_per_capita_USD",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre PBI per capita y matriculas en Europa",
    labels={"PIB_per_capita_USD": "PBI per capita (USD)", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [267]:
# Agrupar promedio por año
df_avg = df_final.groupby("Año")[["Porcentaje_Matriculas", "PIB_per_capita_USD", "Tasa_Natalidad"]].mean().reset_index()

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Matriculas
fig.add_trace(
    go.Scatter(
        x=df_avg["Año"],
        y=df_avg["Porcentaje_Matriculas"],
        name='Porcentaje_Matriculas'
    ),
    secondary_y=False,
)

# PBI
fig.add_trace(
    go.Scatter(
        x=df_avg["Año"],
        y=df_avg["PIB_per_capita_USD"],
        name='PIB_per_capita_USD'
    ),
    secondary_y=True,
)

fig.update_layout(
    title_text="Promedio de matriculas y PBI por año",
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.5)'),
)
fig.update_yaxes(title_text="Promedio Matriculas", secondary_y=False)
fig.update_yaxes(title_text="Promedio PBI per capita", secondary_y=True)

fig.show()

Insights:
- No en todos los continentes podemos afirmar que la relación entre PBI y Porcentaje de Matriculación es directa
- En America del Sur la relación es directamente positiva
- A su vez, se observa que en años más lejanos la matriculación era menor
- Al analizarlo desde los porcentajes observamos una fuerte relación de crecimiento positiva entre el PBI y la Matriculación

## ¿Existe una correlación directa entre la Natalidad y la Matriculación?

In [167]:
# Agrupar promedio por año
df_avg = df_final.groupby("Año")[["Porcentaje_Matriculas", "PIB_per_capita_USD", "Tasa_Natalidad"]].mean().reset_index()

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Matriculas
fig.add_trace(
    go.Scatter(
        x=df_avg["Año"],
        y=df_avg["Porcentaje_Matriculas"],
        name='Porcentaje_Matriculas'
    ),
    secondary_y=False,
)

# PBI
fig.add_trace(
    go.Scatter(
        x=df_avg["Año"],
        y=df_avg["Tasa_Natalidad"],
        name='Tasa_Natalidad'
    ),
    secondary_y=True,
)

fig.update_layout(
    title_text="Promedio de matriculas y tasa de natalidad 19 años previos",
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.5)'),
)
fig.update_yaxes(title_text="Promedio Matriculas", secondary_y=False)
fig.update_yaxes(title_text="Promedio Tasa de Natalidad", secondary_y=True)

fig.show()

In [173]:
# Grafico Scatter
# Europa matriculas y tasa de natalidad
fig = px.scatter(
    df_final[df_final['Continente'] == "Europe"],
    x="Tasa_Natalidad",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre natalidad 19 años previo y matriculas en Europa",
    labels={"Tasa_Natalidad": "Tasa Natalidad", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [174]:
# Grafico Scatter
# Asia matriculas y tasa de natalidad
fig = px.scatter(
    df_final[df_final['Continente'] == "Asia"],
    x="Tasa_Natalidad",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre natalidad 19 años previo y matriculas en Asia",
    labels={"Tasa_Natalidad": "Tasa Natalidad", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [175]:
# Grafico Scatter
# America del Norte matriculas y tasa de natalidad
fig = px.scatter(
    df_final[df_final['Continente'] == "North America"],
    x="Tasa_Natalidad",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre natalidad 19 años previo y matriculas en America del Norte",
    labels={"Tasa_Natalidad": "Tasa Natalidad", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [176]:
# Grafico Scatter
# America del Sur matriculas y tasa de natalidad
fig = px.scatter(
    df_final[df_final['Continente'] == "South America"],
    x="Tasa_Natalidad",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre natalidad 19 años previo y matriculas en America del Sur",
    labels={"Tasa_Natalidad": "Tasa Natalidad", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [177]:
# Grafico Scatter
# Oceania matriculas y tasa de natalidad
fig = px.scatter(
    df_final[df_final['Continente'] == "Oceania"],
    x="Tasa_Natalidad",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre natalidad 19 años previo y matriculas en Oceania",
    labels={"Tasa_Natalidad": "Tasa Natalidad", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

In [178]:
# Grafico Scatter
# Oceania matriculas y tasa de natalidad
fig = px.scatter(
    df_final[df_final['Continente'] == "Africa"],
    x="Tasa_Natalidad",
    y="Porcentaje_Matriculas",
    color="Año",
    title="Relación entre natalidad 19 años previo y matriculas en Africa",
    labels={"Tasa_Natalidad": "Tasa Natalidad", "Porcentaje_Matriculas": "Porcentaje Matriculas"},
    hover_data=["Pais", "Año"],
    trendline="ols"
)
fig.show()

Insights:
- Se observa lo opuesto al PBI per capita, hay una clara disminución de la tasa de natalidad a lo largo de los años y a pesar de eso la matriculación aumenta
- Teniendo en cuenta que la tasa de natalidad es un calculo sobre la población existente, ¿Cambiaría el analisis si se hiciera con la cantidad de nacimientos en vez de la tasa?

## ¿Qué factor tiene mayor influencia?

In [179]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Matriculas
fig.add_trace(
    go.Scatter(
        x=df_avg["Año"],
        y=df_avg["PIB_per_capita_USD"],
        name="Porcentaje Matriculas"
    ),
    secondary_y=False
)

# PBI
fig.add_trace(
    go.Scatter(
        x=df_avg["Año"],
        y=df_avg["Porcentaje_Matriculas"],
        name="PBI per capita",
        line=dict(color="green")
    ),
    secondary_y=True
)

# Natalidad
fig.add_trace(
    go.Scatter(
        x=df_avg["Año"],
        y=df_avg["Tasa_Natalidad"],
        name="Tasa de Natalidad",
        line=dict(color="orange", dash="dash")
    ),
    secondary_y=True
)

fig.update_layout(
    title_text="Promedio de matriculas, PBI y natalidad por año",
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.5)')
)

fig.update_yaxes(title_text="PBI per capita", secondary_y=False)
fig.update_yaxes(title_text="Promedio Matriculas / Tasa Natalidad", secondary_y=True)

fig.show()

Insights:
- El PBI parecería tener una relación directa con el porcentaje de matriculación, aunque pudimos observar que no siempre es así
- La tasa de natalidad pareceria no tener impacto en el porcentaje de matriculación
- ¿Sería más significativo contar con la cantidad exacta de natalidad en vez de la tasa?
- ¿Hay algún otro factos que no se haya tenido en cuenta? Como mayor cantidad de espacios para acceder a una educación superior con el correr de los años.

#